In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
import torch
import os
import torchvision
import PIL
import cv2 

In [ ]:
from torch.utils.data import Dataset,DataLoader
from torch import nn

In [ ]:
from torchvision import transforms

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((96,96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[ 0.5],
    std=[ 0.5])
])

In [ ]:
train_set=torchvision.datasets.ImageFolder('./chest_xray/train/',transform=transform)
test_set=torchvision.datasets.ImageFolder('./chest_xray/test/',transform=transform)
val_set=torchvision.datasets.ImageFolder('./chest_xray/val/',transform=transform)

In [ ]:
from PIL import Image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[ 0.5])
])

# 加载图像并进行预处理
image_path = './chest_xray/my/'
image_names = os.listdir(image_path)
image_name = image_names[0]

In [ ]:
image = Image.open(image_path+image_name)

In [ ]:
print(image)

In [ ]:
image_tensor = transform(image)

In [ ]:
image_tensor.shape

In [ ]:
# 将图像输入模型并得到预测结果
with torch.no_grad():
    output = model(image_tensor)

# 将输出转换为预测类别
_, predicted = torch.max(output.data, 1)
print('预测结果:', predicted.item())

In [ ]:
my_set = MyDataset()

In [ ]:
train_loader=DataLoader(
    dataset=train_set,
    batch_size=32,
)

In [ ]:
test_loader=DataLoader(
    dataset=test_set,
    batch_size=32,
)

In [ ]:
val_loader=DataLoader(
    dataset=val_set,
    batch_size=32,
)

In [ ]:
imgs,labels=next(iter(val_loader))

In [ ]:
id_to_class = dict((v, k) for k, v in train_set.class_to_idx.items())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(16, 10))
for i, (img, label) in enumerate(zip(imgs[:12], labels[:12])):
    img = (img.permute(1, 2, 0).numpy() + 1)/2
    plt.subplot(3, 4, i+1)
    plt.title(id_to_class.get(label.item()))
    plt.imshow(img)

In [ ]:
model=torchvision.models.vgg16(weights=None)

In [ ]:
model.features[0]=nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [ ]:
# for param in model.features.parameters():
#     param.requires_grad=False

In [ ]:
model.classifier[-1].out_features=2

In [ ]:
model

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
import time

def train(model, optimizer, loss_fn, train_dl, val_dl, epochs=20, device='cuda'):
   
    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    history = {} # Collects per-epoch loss and acc like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    history['acc'] = []
    history['val_acc'] = []

    start_time_sec = time.time()
    model.to(device)
    for epoch in range(epochs):

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        model.train()
        train_loss         = 0.0
        num_train_correct  = 0
        num_train_examples = 0

        for batch in train_dl:

            optimizer.zero_grad()

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            print(x.shape,y.shape)
            yhat = model(x)
            loss = loss_fn(yhat, y)

            loss.backward()
            optimizer.step()

            train_loss         += loss.data.item() * x.size(0)
            num_train_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_train_examples += x.shape[0]

        train_acc   = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        val_loss       = 0.0
        num_val_correct  = 0
        num_val_examples = 0

        for batch in val_dl:

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            loss = loss_fn(yhat, y)

            val_loss         += loss.data.item() * x.size(0)
            num_val_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_val_examples += y.shape[0]

        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_dl.dataset)


        print('Epoch %3d/%3d, train loss: %5.2f, train acc: %5.2f, val loss: %5.2f, val acc: %5.2f' % \
              (epoch+1, epochs, train_loss, train_acc, val_loss, val_acc))

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['acc'].append(train_acc)
        history['val_acc'].append(val_acc)

    # END OF TRAINING LOOP
    end_time_sec       = time.time()
    total_time_sec     = end_time_sec - start_time_sec
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))

    plt.figure(dpi=150)
    plt.plot(range(1,epochs+1),history['loss'],label='train_loss')
    plt.plot(range(1,epochs+1),history['val_loss'],label='test_loss')
    plt.title("Loss")
    plt.legend()
    plt.show()

    plt.figure(dpi=150)
    plt.plot(range(1,epochs+1),history['acc'],label='train_acc')
    plt.plot(range(1,epochs+1),history['val_acc'],label='test_acc')
    plt.title("Accuracy")
    plt.legend()
    plt.show()
    return history
     

In [ ]:
history=train(model, optim, loss_fn, train_loader, test_loader, epochs=20, device='cpu')